In [1]:
import sys

In [2]:
print("version:", sys.version)
print("environment:", sys.prefix)

version: 3.10.13 | packaged by conda-forge | (main, Oct 26 2023, 18:07:37) [GCC 12.3.0]
environment: /home/aclexp/mambaforge/envs/quanta


In [3]:
import re

In [4]:
import os
import copy
import json
import numpy as np
import pandas as pd
from itertools import product

In [5]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA

In [6]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [7]:
DF = pd.read_csv(os.path.join("rawdata", "DATA_ses-01_2024-12-09.csv"))
DF.rename(columns={"BASIC_INFO_ID": "ID"}, inplace=True)

inclusion_df = pd.read_csv(os.path.join("rawdata", "InclusionList_ses-01.csv"))
inclusion_df = inclusion_df.query("MRI == 1")

DF = pd.merge(DF, inclusion_df[["ID"]], on="ID", how='inner')

## Select features by specified domains and approaches

In [8]:
included_features = [ 
    col for col in DF.columns
    if any( domain in col for domain in ["STRUCTURE", "MOTOR", "MEMORY", "LANGUAGE"] )
    and any( approach in col for approach in ["MRI", "BEH", "EEG"] )
    and "RESTING" not in col
]

In [9]:
X = DF.loc[:, included_features]

In [10]:
len(X.columns)

1457

In [11]:
# cat_f = categorize_features(X.columns)

# for f in sorted(cat_f.keys()):
#     print(f"# {f}: {len(cat_f[f])}")

## Remove features with too many missing values

In [12]:
n_subjs = len(X)
na_rates = pd.Series(X.isnull().sum() / n_subjs)

Q1 = na_rates.quantile(.25)
Q3 = na_rates.quantile(.75)
IQR = Q3 - Q1
outliers = na_rates[na_rates > (Q3 + IQR * 1.5)]

print(f"{len(outliers)} out of {len(na_rates)} features have too many missing values (> {round(Q3 + IQR * 1.5, 2)}).")

114 out of 1457 features have too many missing values (> 0.14).


In [13]:
X_cleaned = X.drop(columns=outliers.index)

In [14]:
len(X_cleaned.columns)

1343

## Fill missing values with median and standardize features

In [15]:
imputer = SimpleImputer(strategy="median")
X_imputed = pd.DataFrame(imputer.fit_transform(X_cleaned), columns=X_cleaned.columns)

In [16]:
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X_imputed), columns=X_imputed.columns)

## Calculate VIF 
##### See: https://stats.stackexchange.com/questions/461141/is-it-advisable-to-impute-missing-values-and-scale-features-before-computing-the
##### Also: https://stackoverflow.com/questions/42658379/variance-inflation-factor-in-python

In [17]:
# def variance_inflation_factor(X: np.ndarray, idx: int):
#     mask = np.arange(X.shape[1]) != idx
#     X_i = X[:, idx]
#     X_else = X[:, mask]
#     reg = LinearRegression(fit_intercept=True).fit(X_else, X_i)
#     Xi_pred = reg.predict(X_else)
#     R_sq = reg.score(X_else, X_i)
#     # ss_tot = np.sum((X_i - np.mean(X_i)) ** 2)
#     # ss_res = np.sum((X_i - Xi_pred) ** 2)
#     # R_sq = 1 - (ss_res / ss_tot)
#     vif = 1 / max(1 - R_sq, 1e-16)
#     return vif

In [18]:
# vif_values = pd.Series(
#     [ variance_inflation_factor(X_scaled.values, i) for i in range(X_scaled.shape[1]) ], 
#     index=X_scaled.columns, 
#     name='VIF'
# )

In [19]:
# from statsmodels.stats.outliers_influence import variance_inflation_factor
# from statsmodels.tools.tools import add_constant

In [20]:
# X_ = add_constant(X_scaled)
# vif_values = pd.Series(
#     [ variance_inflation_factor(X_.values, i) for i in range(X_.shape[1]) ], 
#     index=X_.columns
# )

In [21]:
# vifs = pd.Series(
#     np.linalg.inv(X_scaled.corr().to_numpy()).diagonal(), # pseudo-inverse
#     index=X_scaled.columns, 
#     name='VIF'
# )

In [22]:
# vifs = vifs.sort_values(by="VIF", ascending=False)

In [23]:
# vifs[vifs.abs() < 5]

## Correlation matrix

In [36]:
X_cormat = X_scaled.corr()
X_corr = X_cormat.stack().reset_index()
X_corr.columns = ['FEATURE_1', 'FEATURE_2', 'CORRELATION']
print(X_corr.describe().round(3))

       CORRELATION
count  1803649.000
mean         0.028
std          0.134
min         -0.918
25%         -0.043
50%          0.008
75%          0.066
max          1.000


In [37]:
(X_corr['CORRELATION'] > .8).sum() / len(X_corr)

0.0023247316966882136

In [25]:
X_cormat.to_csv(os.path.join("derivatives", "Feature cormat.csv"))

In [26]:
# sns.heatmap(X_cormat, annot=True, cmap='coolwarm', fmt=".2f", linewidths=.5)

## Categorize features based on knowledge

In [27]:
def categorize_features(included_features):
    categorized_features = {}
    
    for feature in included_features:
        if feature.startswith("STRUCTURE"):
            category, hemi, area, measure = feature.split("_")[3::]
            if category == "NULL":
                f = f"STR ({category}, {measure})"
            else:
                f = f"STR ({category}, {hemi[0]}, {measure})"
            
        else:
            domain, task, measure, condition = feature.split("_")[:4]
            measure = "fMRI" if measure == "MRI" else measure                

            if domain == "LANGUAGE":
                f = f"{measure} ({domain.lower()})"
                
            elif (measure == "EEG") and (task == "OSPAN") and ("Diff" in condition):
                f = f"{measure} ({domain.lower()}, {task.lower()} diff)"
                
            elif (measure == "EEG") and (task == "GOFITTS"):
                suffix = re.sub(r"[0-9]+", "", condition).replace("ID", "").replace("W", "").replace("Slope", " slope")
                f = f"{measure} ({domain.lower()}, {task.lower()} {suffix.lower()})"
            
            else:
                f = f"{measure} ({domain.lower()}, {task.lower()})"

        if f in categorized_features.keys():
            categorized_features[f].append(feature)
        else:
            categorized_features.update({f: [feature]})
    
    return categorized_features

In [28]:
categorized_features = categorize_features(X_cleaned.columns)

In [29]:
for f in sorted(categorized_features.keys()):
    print(f"# {f}: {len(categorized_features[f])}")

# BEH (language): 14
# BEH (memory, exclusion): 54
# BEH (memory, mst): 33
# BEH (memory, ospan): 20
# BEH (motor, bilpress): 6
# BEH (motor, gforce): 16
# BEH (motor, gofitts): 24
# EEG (memory, exclusion): 60
# EEG (memory, ospan diff): 48
# EEG (memory, ospan): 72
# EEG (motor, bilpress): 32
# EEG (motor, gofitts cue slope): 36
# EEG (motor, gofitts cue): 72
# EEG (motor, gofitts go slope): 36
# EEG (motor, gofitts go): 72
# STR (GM, L, FA): 34
# STR (GM, L, ThickAvg): 74
# STR (GM, L, ThickStd): 74
# STR (GM, L, VOLUME): 74
# STR (GM, R, FA): 34
# STR (GM, R, ThickAvg): 74
# STR (GM, R, ThickStd): 74
# STR (GM, R, VOLUME): 74
# STR (NULL, FA): 36
# STR (WM, L, FA): 34
# STR (WM, L, VOLUME): 34
# STR (WM, R, FA): 34
# STR (WM, R, VOLUME): 34
# fMRI (motor, gforce): 64


In [30]:
np.array([ len(v) for v in categorized_features.values() ]).sum()

1343

In [31]:
categorized_outliers = categorize_features(outliers.index)

for f in sorted(categorized_outliers.keys()):
    print(f"# {f}: {len(categorized_outliers[f])}")

# BEH (memory, exclusion): 6
# BEH (memory, mst): 2
# STR (NULL, FA): 1
# fMRI (memory, mst): 105


In [32]:
X_splited = {}
for f in categorized_features.keys():
    X_splited[f] = X_scaled.loc[:, categorized_features[f]]

## Combining variables using PCA 

In [33]:
## https://stackoverflow.com/questions/62303782/is-there-a-way-to-conduct-a-parallel-analysis-in-python
## https://www.statstodo.com/ParallelAnalysis.php

def parallel_analysis(X, n_iter=100, seed=42):
    n_components = X.shape[1] - 1
    pca = PCA(n_components, random_state=seed)
    X_transformed = pca.fit_transform(X)
    eigenvalues = pca.explained_variance_ratio_

    random_eigenvalues = np.zeros(shape=(n_iter, n_components))
    for i in range(n_iter): # Monte Carlo simulation
        X_r = np.random.normal(loc=0, scale=1, size=X.shape) # random data
        pca_r = PCA(n_components, random_state=42)
        X_r_transformed = pca_r.fit_transform(X_r)
        random_eigenvalues[i, :] = pca_r.explained_variance_ratio_
    
    rand_eigv_mean = random_eigenvalues.mean(axis=0)
    rand_eigv_std = random_eigenvalues.std(axis=0)
    thresholds = rand_eigv_mean + rand_eigv_std * 1.64 # the 95th percentile
    n_retained = max(np.argwhere(eigenvalues > thresholds)) + 1

    return n_retained, eigenvalues, random_eigenvalues

In [34]:
n_iter = 100
n_retained_comps = {}

for f, X_n in X_splited.items():
    n_components, eigv_raw, eigv_rand = parallel_analysis(X_n)
    n_retained_comps[f] = n_components[0]

In [35]:
n_retained_comps

{'STR (GM, L, VOLUME)': 5,
 'STR (GM, L, ThickAvg)': 5,
 'STR (GM, L, ThickStd)': 4,
 'STR (GM, R, VOLUME)': 5,
 'STR (GM, R, ThickAvg)': 6,
 'STR (GM, R, ThickStd)': 5,
 'STR (WM, L, VOLUME)': 2,
 'STR (WM, R, VOLUME)': 3,
 'STR (NULL, FA)': 4,
 'STR (GM, L, FA)': 5,
 'STR (GM, R, FA)': 5,
 'STR (WM, L, FA)': 2,
 'STR (WM, R, FA)': 3,
 'BEH (language)': 4,
 'BEH (memory, exclusion)': 7,
 'BEH (memory, ospan)': 3,
 'BEH (memory, mst)': 6,
 'EEG (memory, exclusion)': 11,
 'EEG (memory, ospan)': 9,
 'EEG (memory, ospan diff)': 8,
 'BEH (motor, gforce)': 3,
 'fMRI (motor, gforce)': 8,
 'EEG (motor, gofitts cue slope)': 12,
 'EEG (motor, gofitts go slope)': 9,
 'EEG (motor, gofitts cue)': 7,
 'EEG (motor, gofitts go)': 7,
 'BEH (motor, gofitts)': 4,
 'BEH (motor, bilpress)': 2,
 'EEG (motor, bilpress)': 5}

## Else

##### Mundfrom, D. J., Shaw, D. G., & Ke, T. L. (2005). Minimum sample size recommendations for conducting factor analyses. https://doi.org/10.1207/s15327574ijt0502_4

In [38]:
sub_DF_list = [
    DF[DF["BASIC_INFO_AGE"].between(lb, ub)] for lb, ub in [(0, 44), (45, np.inf)]
]

print(len(DF) / 5)

for sub_DF in sub_DF_list:
    print(len(sub_DF) / 5)

84.6
36.0
48.6


In [40]:
for sub_DF in sub_DF_list:
    print(len(sub_DF) / 3)

60.0
81.0


In [39]:
# set([ "_".join(f.split("_")[:3]) for f in DF.columns ])